In [ ]:
from IPython.display import clear_output
import os, glob


# Downloading teh dataset /
#!gdown --id 1iWX_NOqu9PWldq-_4ti3iKsJugXLec0- 
!unrar x round2data.rar; rm round2data.rar;
clear_output()

!rm -r sample_data

# Renaming raw data folder to remove space. Trust me, it makes life a lot easier :D 
%mv 'Final Train Dataset' train_data_raw


rm: cannot remove 'sample_data': No such file or directory
mv: cannot stat 'Final Train Dataset': No such file or directory


In [ ]:
corrupt_files = ['231.jpg', '231.xml', 'Pias (359).PNG','Pias (359).xml', 'Pias (360).PNG', 'Pias (360).xml']

%cd /content/train_data_raw/

for file in corrupt_files:
    file_path = os.path.join('/content/train_data_raw/', file)  
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f'{file} is removed successfully')
    else:
        print(f'{file} is already deleted')

%cd /content/

/content/train_data_raw
231.jpg is removed successfully
231.xml is removed successfully
Pias (359).PNG is removed successfully
Pias (359).xml is removed successfully
Pias (360).PNG is removed successfully
Pias (360).xml is removed successfully
/content


In [ ]:
"""
Thanks to @bjornstenger for his excellent code for converting the code from XML format to .txt format
here is the original link of this cell of code.
Original Link: https://github.com/bjornstenger/xml2yolo/blob/master/convert.py 
"""

from xml.dom import minidom

# Remember these number assigned. These are the label indexes which will be used in the training process.
# Feel free to unfold to see what's inside 
lut={"ambulance": 0,
     "auto rickshaw": 1,
     "bicycle": 2,
     "bus": 3,
     "car": 4,
     "garbagevan": 5,
     "human hauler": 6,
     "minibus": 7,
     "minivan": 8,
     "motorbike": 9,
     "pickup": 10,
     "army vehicle": 11,
     "policecar": 12,
     "rickshaw": 13,
     "scooter": 14,
     "suv": 15,
     "taxi": 16,
     "three wheelers (CNG)": 17,
     "truck": 18,
     "van": 19,
     "wheelbarrow": 20
     }

label_count ={}

print(f'Object Names: {list(lut.keys())}' )

def convert_coordinates(size, box):
    """
    This function converts the coordinates. 
    box: (xmin, ymin, xmax, ymax)
    size: (width, height)

    returns a touple where (x, y, height, width) of the boundary box
    """
    dw = 1.0/size[0]
    dh = 1.0/size[1]
    x = (box[0]+box[1])/2.0
    y = (box[2]+box[3])/2.0
    w = box[1]-box[0]
    h = box[3]-box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)


def convert_xml2yolo(filelist, lut ):
    """
    filelist: list of .xml file paths to convert to .txt file
    lut: a dictionary containing class_name to class_index mapping
    """
    for fname in filelist:
        xmldoc = minidom.parse(fname)
        fname_out = (fname[:-4]+'.txt')
        print(fname_out)

        with open(fname_out, "w") as f:
            # print(f'processing{fname}')

            itemlist = xmldoc.getElementsByTagName('object')
            size = xmldoc.getElementsByTagName('size')[0]
            width = int((size.getElementsByTagName('width')[0]).firstChild.data)
            height = int((size.getElementsByTagName('height')[0]).firstChild.data)

            for item in itemlist:
                # get class label
                classid =  (item.getElementsByTagName('name')[0]).firstChild.data
                if classid in lut:
                    label_str = str(lut[classid])
                else:
                    label_str = "-1"
                    print ("warning: label '%s' not in look-up table for file '%s'" % classid, fname )
                # get bbox coordinates
                xmin = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('xmin')[0]).firstChild.data
                ymin = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('ymin')[0]).firstChild.data
                xmax = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('xmax')[0]).firstChild.data
                ymax = ((item.getElementsByTagName('bndbox')[0]).getElementsByTagName('ymax')[0]).firstChild.data
                b = (float(xmin), float(xmax), float(ymin), float(ymax))
                bb = convert_coordinates((width,height), b)
                #print(bb)

                label_count[classid] = label_count.get(classid, 0) + 1

                f.write(label_str + " " + " ".join([("%.6f" % a) for a in bb]) + '\n')
        # print ("wrote %s" % fname_out)


Object Names: ['ambulance', 'auto rickshaw', 'bicycle', 'bus', 'car', 'garbagevan', 'human hauler', 'minibus', 'minivan', 'motorbike', 'pickup', 'army vehicle', 'policecar', 'rickshaw', 'scooter', 'suv', 'taxi', 'three wheelers (CNG)', 'truck', 'van', 'wheelbarrow']


Now Let us create the path list of the `.xml` files to convert them to `.txt` files. 

In [ ]:
# Reading Image file paths
formats = ['jpg', 'jpeg', 'JPG', 'png']
image_file_list = []
for format in formats:
    image_file_list.extend(glob.glob(f'/content/train_data_raw/*.{format}'))

# Reading XML label file paths
label_file_list_xml = glob.glob('/content/train_data_raw/*.xml')

print(f'Image files found: {len(image_file_list)} \nLabel files found: { len(label_file_list_xml)}')

Image files found: 3813 
Label files found: 3813


In [ ]:
# Converting .xml file to .txt file
convert_xml2yolo(label_file_list_xml, lut)
label_file_list_txt = glob.glob('/content/train_data_raw/*.txt')
print(f'XML --> TXT files: {len(label_file_list_txt)}')

In [ ]:
label_count

{'ambulance': 188,
 'army vehicle': 141,
 'auto rickshaw': 394,
 'bicycle': 617,
 'bus': 3645,
 'car': 5918,
 'garbagevan': 132,
 'human hauler': 285,
 'minibus': 142,
 'minivan': 958,
 'motorbike': 2443,
 'pickup': 1389,
 'policecar': 206,
 'rickshaw': 3930,
 'scooter': 157,
 'suv': 1053,
 'taxi': 426,
 'three wheelers (CNG)': 3088,
 'truck': 1635,
 'van': 801,
 'wheelbarrow': 280}

## Data Visualization: 
Let us have a look at the existance of the labels in the dataset. 

In [ ]:
import pandas as pd
import plotly.express as px

# DataFrame Generation
df = pd.DataFrame({'labels': label_count.keys(), 'count': label_count.values()})
df.columns = ['labels', 'count']
df.sort_values(['count'], ascending = False, inplace =True)
df.head()

# Plotting
fig = px.bar(df, x="labels", y='count',  color="count",
    orientation='v', 
    title='Frequency of the Labels in Dhaka.ai-2020 Challenge', 
    color_continuous_scale=px.colors.sequential.Viridis_r
)
fig.update_layout(title_x=0.5, xaxis_title = 'Labels', yaxis_title = 'Label Count')
fig.update_xaxes(tickangle=60)
fig.show()

In [ ]:
from PIL import Image
img_sizes = {}

for fname in image_file_list:
    img = Image.open(fname)
    img_sizes[img.size] = img_sizes.get(img.size, 0) +1 
img_sizes

{(1024, 1024): 3813}

In [ ]:
def resize_images(file_list, width = 1024, height = 1024, overwrite = True, save_dir = ''):
    total_files = len(file_list)
    idx = 1
    for path in file_list:
        img = Image.open(path)
        img_resized = img.resize((width, height))
        if overwrite:
            img_resized.save(path)
            filename = path.split('/')[-1] 
            print(f"{idx}/{total_files}: {filename} {img.size}--> ({width}x{height})")
        else:
            filename = path.split('/')[-1]
            img_resized.save(save_dir + filename)
            print(f'{filename} saved to {save_dir}')
        idx +=1
    clear_output()

In [ ]:
resize_images(image_file_list , overwrite= True)

In [ ]:
!gdown --id 1nPrI9CUXar7ht-4vpPASsqAkZm3jOrDj
!unrar x /content/round2v5ew.rar

In [ ]:
#!mv /content/round2v5ew/* /content/
!mv /content/valid/* /content/train/

In [ ]:
import glob
formats = ['jpg', 'jpeg', 'JPG', 'png']
image_file_list = []
for format in formats:
    image_file_list.extend(glob.glob(f'/content/train/*.{format}'))

# Reading XML label file paths
label_file_list_txt = glob.glob('/content/train/*.txt')

print(f'Image files found: {len(image_file_list)} \nLabel files found: { len(label_file_list_xml)}')

Image files found: 3819 
Label files found: 3807


We don't have to resize the test images because they are already resized to our desired format. 

In [ ]:
import random
random.seed(999)

#randomply selecting the index of the files
valid_set_index = random.sample(range(len(image_file_list)), 500)
len(set(image_file_list)), len(set(label_file_list_txt)), len(valid_set_index)

image_file_list = sorted(image_file_list)
label_file_list_txt = sorted(label_file_list_txt)

# sanity check of the image files and labels being in the same order
print('Checking files concurrency')
print(image_file_list[:5])
print(label_file_list_txt[:5])

# code to separate train and validation set
valid_selected_images = []
valid_selected_labels = []

for index in valid_set_index: 
    valid_selected_images.append(image_file_list[index])
    valid_selected_labels.append(label_file_list_txt[index])


print('\n\nChecking files concurrency in validation set')
print(valid_selected_images[:5])
print(valid_selected_labels[:5])

Checking files concurrency
['/content/train/01 (1).jpg', '/content/train/01 (1)_jpg.jpg', '/content/train/01 (1)_jpg233.jpg', '/content/train/01.jpg', '/content/train/02.jpg']
['/content/train/01 (1).txt', '/content/train/01 (1)_jpg.txt', '/content/train/01 (1)_jpg233.txt', '/content/train/01.txt', '/content/train/02.txt']


Checking files concurrency in validation set
['/content/train/Pias (529).jpg', '/content/train/Pias (149).jpg', '/content/train/320px-Going_local_Bangladesh_minibus_(26345425925)_jpg.jpg', '/content/train/sakfjaksjfks.jpg', '/content/train/pt (10).jpg']
['/content/train/Pias (529).txt', '/content/train/Pias (149).txt', '/content/train/320px-Going_local_Bangladesh_minibus_(26345425925)_jpg.txt', '/content/train/sakfjaksjfks.txt', '/content/train/pt (10).txt']


In [ ]:
import shutil
import os
# Creating validation directory
valid_dir = '/content/valid/'

if os.path.exists(valid_dir):
    print(f'Directory {valid_dir} already exists !')
else: 
    os.makedirs(valid_dir)
    print(f"Directory {valid_dir} is created successfully!") 


for idx in range(len(valid_selected_images)):
    # moving image files
    mypath = valid_selected_images[idx]
    if os.path.exists(mypath):
        filename = mypath.split('/')[-1]
        shutil.move(mypath , valid_dir + filename)
    else:
        print(f'{mypath} not found')
        
    # moving label files
    mypath = valid_selected_labels[idx]
    if os.path.exists(mypath):
        filename = mypath.split('/')[-1]
        shutil.move(mypath , valid_dir + filename)
    else:
        print(f'{mypath} not found')

Directory /content/valid/ already exists !


Now the remaining images in the `train_data_raw` are actually the train directory. We will rename it to `train`

In [ ]:
!mv train_data_raw train  

In [ ]:
def lookup_image_file_paths(formats, dir):
    """
    This function takes a specified set of formats and directory address to list all the filepaths
    of the desired format in that directory
    """
    filepaths = []
    for format in formats:
        filepaths.extend(glob.glob(f'{dir}*.{format}'))
    return filepaths

def make_txt_file(formats, dir):
    """
    Formats the file names to write in the desired txt file
    """
    filepaths = lookup_image_file_paths(formats, dir)
    
    filenames = [x.split('/')[-1] for x in filepaths]
    txt_file_name = dir.split('/')[-2]

    print(f'{txt_file_name} : {len(filepaths)} images')
    with open(f'/content/metadata/{txt_file_name}.txt', 'w') as outfile:
        for filename in filenames:
            outfile.write(f'data/{txt_file_name}/'+filename+'\n')
        outfile.close()


In [ ]:
train_dir = '/content/train/'
valid_dir = '/content/valid/'
test_dir =  '/content/test/'
!mkdir metadata

# Making the .txt file containing list of images. 
make_txt_file(formats, dir = train_dir)
make_txt_file(formats, dir = test_dir)
make_txt_file(formats, dir = valid_dir)

# Writing the file traffic.names
object_labels = list(lut.keys())
with open('/content/metadata/traffic.names', 'w') as outfile:
    for label in object_labels:
        outfile.write(label + '\n')
    outfile.close()

# Writing the file traffic.data
data_config = f'classes=21\ntrain=train.txt\nvalid=valid.txt\nnames=traffic.names'
with open('/content/metadata/traffic.data', 'w') as outfile:
    outfile.write(data_config + '\n')
    outfile.close()

mkdir: cannot create directory ‘metadata’: File exists
train : 3319 images
test : 450 images
valid : 500 images


NameError: ignored

<a id="7"></a>
## 7. Saving the Processed Dataset:
Now our dataset is ready for using with the starter code given at the host site. But before that we need to save the processed dataset in google drive so that we can later use it in the Starter Code. 

<font color='red'> Now mount your google drive here so that you can export the processed `dhaka-traffic-yolo-v3.zip` to your drive for later use </font> Or you can just download it and again re upload it to the Starter Notebook in Google Colab. 

In [ ]:
!zip -r round2v5ew3.zip train test valid
clear_output()

Streaming output truncated to the last 5000 lines.
  adding: train/Dipto_ 72.jpg (deflated 1%)
  adding: train/Dipto_531.txt (deflated 46%)
  adding: train/Navid_437.xml (deflated 79%)
  adding: train/Navid_532.xml (deflated 81%)
  adding: train/Navid_284.xml (deflated 68%)
  adding: train/163.jpg (deflated 1%)
  adding: train/220.xml (deflated 92%)
  adding: train/Dipto_ 341.xml (deflated 77%)
  adding: train/Dipto_581.txt (deflated 8%)
  adding: train/Navid_486.txt (deflated 55%)
  adding: train/Dipto_396.txt (deflated 39%)
  adding: train/Suzuki-Lets-Scooter-review-by-Mir-Rabbi.txt (deflated 8%)
  adding: train/Pias (443).txt (deflated 8%)
  adding: train/amb6.xml (deflated 62%)
  adding: train/Dipto_391.txt (deflated 41%)
  adding: train/amb6_jpg303.txt (deflated 25%)
  adding: train/Navid_748.xml (deflated 79%)
  adding: train/Navid_537.xml (deflated 80%)
  adding: train/Navid_388.xml (deflated 74%)
  adding: train/Pias (471).jpg (deflated 3%)
  adding: train/Pias (339).xml (defla

NameError: ignored

In [ ]:
!cp round2v5ew3.zip '/content/drive/MyDrive/Colab Notebooks/Dhaka ai/round2v5ew3.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Now we are done with pre-processing the data and we are ready to put the whole dataset to the training of Yolo V3 in the Starter Code given on the host website. 

Thank you and good luck!





